In [24]:
import random
import numpy as np
import operator
import math
import pandas as pd
import time
import os
import dill
from sympy import *
from sympy.parsing import sympy_parser
from mpl_toolkits.mplot3d import Axes3D
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from scoop import futures
from pandas import DataFrame

from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import multiprocessing as mp

# def plot_pred(hof):
#     func = toolbox.compile(expr=hof[0])
#     y_hat = []
#     y = []
#     y_r = []
#     for a, i in enumerate(t):
#         y_hat.append(func(i))
#         y.append(p[a])
#         y_r.append(p_r[a])
        
#     from matplotlib import pyplot as plt
#     plt.plot(t,y)
#     plt.plot(t, y_hat)
#     plt.plot(t, y_r)
#     plt.xlabel('time')  # x-axis
#     plt.ylabel('x(t)')  # y-axis
#     plt.title('Data')  # title
#     plt.grid()  # grid
#     plt.legend(['with error','predicted','real'])
#     plt.show()  # plot show

###################### Define functions############################
def safeDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1
    
def sqrt(left):
    if left>=0:
        return left**0.5
    else:
        return 0
    
def expe(left):
    if left>=0:
        return np.exp(-left)
    else:
        return 0


pset = gp.PrimitiveSet("MAIN", 8)
pset.addPrimitive(operator.add, 2)
#pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(expe, 1)
#set.addPrimitive(safeDiv, 2)
pset.addPrimitive(sqrt,1)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

pset.renameArguments(ARG0='x')
pset.renameArguments(ARG1='y')
pset.renameArguments(ARG2='c1')
pset.renameArguments(ARG3='c2')
pset.renameArguments(ARG4='c3')
pset.renameArguments(ARG5='c4')
pset.renameArguments(ARG6='c5')
pset.renameArguments(ARG7='c6')


###################################################################
pool = mp.Pool()

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
#toolbox.register("map", futures.map)

#########################input DATA#################################

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)

    # Evaluate the mean squared error between the expression
    sqerrors = ((func(x,y,c1,c2,c3,c4,c5,c6) - f[t])**2 for t,x,y,c1,c2,c3,c4,c5,c6 in points)

    try:
        return max(sqerrors),
    except IndexError:
        return max(sqerrors),
    
toolbox.register("map", pool.map)

t = list(np.arange(0,365))

g = pd.read_csv('temp.csv')
x = g['x']
y = g['y']
dx = g['dx']
dy = g['dy']

f = {}
for i in range(len(t)):
    f[t[i]] = 0.5*dx[i]**2 + 0.5*dy[i]**2
    
    
pts=[]

for i in range(len(t)):
    pts.append((t[i],x[i],y[i],1,2,3,4,5,0.5))
  
    

    
toolbox.register("evaluate", evalSymbReg, points=pts)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))

def main():
    pop = toolbox.population(n=20000)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    
    
    #############################################C.O, Muta,Gener##########
    try:
        pop, log = algorithms.eaSimple(pop, toolbox, 0.8, 0.8, 1000, stats=mstats,
                                       halloffame=hof, verbose=True)
        print (hof[0])

        return hof
    
    except MemoryError:
        print (hof[0])

        return hof
    
    except KeyboardInterrupt:
        print (hof[0])

        return hof



if __name__ == "__main__":    
     hof = main()




/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ban94gy/ban/local/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


   	      	          fitness          	    size   
   	      	---------------------------	-----------
gen	nevals	max        	min        	max	min
0  	20000 	3.11321e+28	1.77241e+12	14 	2  
1  	19204 	6.17235e+32	1.77054e+12	24 	2  
2  	19235 	5.4307e+43 	1.46293e+12	31 	2  
3  	19194 	6.66732e+40	1.28525e+12	36 	2  
4  	19219 	5.60312e+41	1.20765e+12	32 	2  
5  	19189 	1.13871e+35	1.23168e+12	32 	2  
6  	19215 	2.23962e+40	1.21246e+12	37 	2  
7  	19211 	8.26217e+34	1.20713e+12	36 	2  
8  	19211 	3.30187e+34	1.20713e+12	39 	2  
9  	19233 	2.66691e+39	1.19431e+12	40 	2  
10 	19216 	1.21438e+46	1.19431e+12	41 	2  
11 	19229 	2.30843e+33	1.08097e+12	46 	2  
12 	19204 	1.80562e+33	1.07874e+12	42 	2  
13 	19255 	6.21711e+34	1.07874e+12	47 	2  
14 	19221 	3.30187e+36	1.07477e+12	47 	2  
15 	19229 	1.07397e+37	1.07477e+12	45 	2  
16 	19177 	5.69357e+33	1.06648e+12	47 	2  
17 	19177 	2.66695e+39	1.0631e+12 	46 	2  
18 	19272 	1.66682e+40	1.06154e+12	45 	2  
19 	19205 	4.51093e+32	1.06154e+12	48 

188	19176 	6.32619e+34	8.09028e+11	77 	2  
189	19201 	5.06095e+33	8.08664e+11	77 	2  
190	19144 	7.27699e+34	8.08664e+11	77 	2  
191	19181 	2.26281e+46	8.08664e+11	77 	2  
192	19225 	5.37394e+33	8.08664e+11	79 	2  
193	19180 	1.26524e+33	8.08664e+11	79 	2  
194	19229 	9.41751e+39	8.08294e+11	79 	2  
195	19156 	8.56996e+30	8.08294e+11	79 	2  
196	19198 	1.02484e+35	8.08294e+11	78 	2  
197	19220 	5.1792e+36 	8.08294e+11	77 	2  
198	19241 	1.32254e+35	8.08294e+11	78 	2  
199	19240 	3.10048e+34	8.00055e+11	79 	2  
200	19245 	2.53048e+33	8.00055e+11	78 	2  
201	19222 	5.06095e+33	7.98632e+11	81 	2  
202	19176 	5.33382e+39	7.94895e+11	78 	2  
203	19166 	2.53047e+33	7.92853e+11	80 	2  
204	19148 	3.21288e+36	7.87589e+11	79 	2  
205	19188 	5.45687e+36	7.87589e+11	79 	2  
206	19175 	5.44197e+29	7.84939e+11	79 	2  
207	19174 	2.24561e+37	7.84043e+11	78 	2  
208	19232 	1.94885e+33	7.79483e+11	75 	2  
209	19202 	1.02605e+33	7.79483e+11	74 	2  
210	19261 	1.94885e+33	7.79483e+11	78 	2  
211	19226 	

379	19232 	1.60202e+33	6.4963e+11 	81 	2  
380	19263 	2.29399e+43	6.4963e+11 	87 	2  
381	19205 	1.01219e+34	6.4963e+11 	87 	2  
382	19185 	1.37895e+35	6.4963e+11 	88 	2  
383	19140 	2.67786e+32	6.4963e+11 	87 	2  
384	19167 	1.61952e+35	6.4963e+11 	87 	2  
385	19247 	2.78435e+39	6.4963e+11 	87 	2  
386	19207 	3.20398e+36	6.4963e+11 	82 	2  
387	19249 	1.01219e+34	6.4963e+11 	87 	2  
388	19194 	5.96411e+39	6.4963e+11 	87 	2  
389	19161 	5.99324e+39	6.4963e+11 	87 	2  
390	19176 	2.86665e+39	6.4963e+11 	88 	2  
391	19208 	4.04875e+34	6.4963e+11 	87 	2  
392	19211 	2.38834e+40	6.4963e+11 	87 	2  
393	19172 	8.28705e+37	6.4963e+11 	88 	2  
394	19173 	3.35703e+35	6.4963e+11 	87 	2  
395	19216 	2.83732e+32	6.4963e+11 	87 	2  
396	19205 	2.67786e+32	6.4963e+11 	87 	2  
397	19196 	9.90971e+39	6.4963e+11 	87 	2  
398	19209 	1.01219e+34	6.4963e+11 	87 	2  
399	19162 	2.67786e+32	6.4963e+11 	88 	2  
400	19187 	4.22091e+38	6.4963e+11 	87 	2  
401	19199 	1.01219e+34	6.4963e+11 	87 	2  
402	19215 	

570	19174 	6.04309e+39	6.47888e+11	89 	2  
571	19191 	3.91738e+38	6.47888e+11	91 	2  
572	19220 	6.04305e+39	6.47888e+11	91 	2  
573	19207 	1.01912e+34	6.47888e+11	91 	2  
574	19266 	1.40248e+44	6.47888e+11	91 	2  
575	19217 	2.66642e+32	6.47888e+11	91 	2  
576	19209 	6.10912e+41	6.47888e+11	91 	2  
577	19197 	7.90349e+34	6.47888e+11	91 	2  
578	19205 	5.40025e+36	6.47888e+11	91 	2  
579	19188 	2.66642e+32	6.47888e+11	90 	2  
580	19169 	2.85407e+32	6.47888e+11	91 	2  
581	19133 	1.37301e+35	6.47888e+11	91 	2  
582	19215 	1.01914e+34	6.47888e+11	91 	2  
583	19171 	5.68929e+33	6.47888e+11	91 	2  
584	19174 	3.10937e+35	6.47888e+11	91 	2  
585	19215 	3.38947e+35	6.47888e+11	91 	2  
586	19197 	9.18018e+34	6.47888e+11	92 	2  
587	19182 	2.77252e+39	6.47888e+11	91 	2  
588	19190 	9.51685e+35	6.47888e+11	91 	2  
589	19211 	5.33649e+35	6.47888e+11	91 	2  
590	19209 	3.39179e+35	6.47888e+11	91 	2  
591	19167 	1.06657e+33	6.47888e+11	91 	2  
592	19190 	2.55936e+35	6.47888e+11	91 	2  
593	19247 	

761	19144 	3.67409e+34	6.47016e+11	95 	2  
762	19255 	2.01798e+35	6.47016e+11	89 	2  
763	19239 	1.01912e+34	6.47016e+11	95 	2  
764	19198 	9.47245e+33	6.47016e+11	95 	2  
765	19129 	1.01912e+34	6.47016e+11	95 	2  
766	19218 	3.46417e+30	6.47016e+11	95 	2  
767	19197 	1.46964e+35	6.47016e+11	95 	2  
768	19221 	1.63059e+35	6.47016e+11	96 	2  
769	19229 	1.04365e+35	6.47016e+11	95 	2  
770	19221 	1.01912e+34	6.47016e+11	95 	2  
771	19210 	1.01912e+34	6.47016e+11	95 	2  
772	19179 	5.49203e+35	6.47016e+11	95 	2  
773	19224 	2.52024e+33	6.47016e+11	95 	2  
774	19179 	1.00899e+36	6.47016e+11	88 	2  
775	19164 	1.43418e+35	6.47016e+11	95 	2  
776	19239 	2.5808e+34 	6.47016e+11	95 	2  
777	19165 	3.67409e+34	6.47016e+11	95 	2  
778	19237 	6.04305e+39	6.47016e+11	95 	2  
779	19212 	1.61296e+33	6.47016e+11	95 	2  
780	19258 	2.14412e+36	6.47016e+11	96 	2  
781	19233 	1.61438e+33	6.47016e+11	88 	2  
782	19165 	9.55362e+36	6.47016e+11	95 	2  
783	19208 	2.66642e+32	6.47016e+11	94 	2  
784	19192 	

952	19182 	1.61296e+33	6.46974e+11	87 	2  
953	19195 	8.35726e+37	6.46974e+11	91 	2  
954	19238 	1.6454e+33 	6.46974e+11	92 	2  
955	19215 	2.14304e+36	6.46974e+11	87 	2  
956	19190 	7.07013e+34	6.46974e+11	92 	2  
957	19144 	4.13867e+33	6.46974e+11	86 	2  
958	19229 	1.0081e+32 	6.46974e+11	87 	2  
959	19185 	1.9517e+35 	6.46974e+11	91 	2  
960	19197 	1.71519e+36	6.46974e+11	91 	2  
961	19137 	6.84907e+39	6.46974e+11	92 	2  
962	19201 	1.47036e+35	6.46974e+11	87 	2  
963	19196 	4.32732e+40	6.46974e+11	91 	2  
964	19199 	4.0765e+34 	6.46974e+11	91 	2  
965	19169 	2.19933e+41	6.46974e+11	91 	2  
966	19211 	2.19577e+36	6.46974e+11	91 	2  
967	19218 	1.71723e+38	6.46974e+11	91 	2  
968	19236 	6.84903e+39	6.46974e+11	91 	2  
969	19214 	1.00909e+34	6.46974e+11	91 	2  
970	19221 	1.61296e+33	6.46974e+11	91 	2  
971	19271 	2.32267e+35	6.46974e+11	92 	2  
972	19214 	1.01912e+34	6.46974e+11	92 	2  
973	19212 	1.47032e+35	6.46974e+11	91 	2  
974	19228 	2.96892e+39	6.46974e+11	91 	2  
975	19187 	

In [ ]:
len(xx)

In [20]:
from sympy import *
from sympy.parsing import sympy_parser


x = Symbol('x')
y = Symbol('y')


def add(a,b):
    return a+b

def expe(a):
    return exp(-a)

def mul(a,b):
    return a*b

def neg(a):
    return -a

def sub(a,b):
    return a-b

def sqrt(a):
    return a**0.5

def safeDiv(a,b):
    if b==0:
        return 1
    else:
        return a/b

expr_dict = {'add':add, 'expe':expe, 'sub':sub, 'mul':mul,'neg':neg,'sqrt':sqrt, 'safeDiv': safeDiv, 'c1':1, 'c2':2, 'c3':3, 'c4':4,'c5':5, 'c6':0.5}

d = sympy_parser.parse_expr(str(hof[0]), local_dict=expr_dict)
d

-x**3*(-400*x - 4)*(y + 1)